# Testing crew.kickoff() - Hello World Example

In [1]:
from openai import OpenAI
from langchain_openai import ChatOpenAI


In [2]:
from crewai import Crew, Agent, Task
from crewai import LLM

In [3]:
def step_callback_function(output):
    print('Step Callback function called')
    print(f"step callback: {output}")

def task_callback_function(output):
    print('Task_Callback function called')
    print(f"task callback: {output}")

In [4]:
API_BASE_URL = 'http://192.168.30.100:8000/'
LITELLM_MASTER_KEY= 'sk-0Mev2fJcwfLy1CMEUHxYxQ'
OPENAI_API_BASE="http://192.168.30.100:8000/v1"

In [5]:
from langchain_community.chat_models import ChatLiteLLM
from langchain.callbacks.manager import CallbackManager
import openai
import requests
import tiktoken
from langchain.callbacks.base import BaseCallbackHandler

class TokenCounterCallback(BaseCallbackHandler):
    def __init__(self, tokenizer):
        self.llm = None
        self.input_tokens = 0
        self.output_tokens = 0
        self.tokenizer = tokenizer

    def on_llm_start(self, serialized, prompts, **kwargs):
        for prompt in prompts:
            self.input_tokens += len(self.tokenizer.encode(prompt, disallowed_special=()))

    def on_llm_end(self, response, **kwargs):
        for generation in response.generations:
            for result in generation:
                self.output_tokens += len(self.tokenizer.encode(result.text, disallowed_special=()))

def get_models():
    data=""
    #data = cache.get('models')
    if not data:
        url = f'{API_BASE_URL}/models'
        headers = {'accept': 'application/json', 'Authorization': f'Bearer {LITELLM_MASTER_KEY}'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()['data']
            # Sort the data by 'id' in ascending order
            data.sort(key=lambda x: x['id'])
        else:
            return None
    return [item['id'] for item in data]
def get_llm(model_name:str, temperature=0.0):

    tokenizer=tiktoken.get_encoding("cl100k_base")
    token_counter_callback = TokenCounterCallback(tokenizer)
     
    callback_manager = CallbackManager([token_counter_callback])  

    llm = ChatOpenAI(model=model_name, 
                     base_url=API_BASE_URL, 
                     api_key=LITELLM_MASTER_KEY, 
                     temperature=temperature, 
                     callbacks=callback_manager)

    token_counter_callback.llm = llm
    return llm, token_counter_callback

def get_crew_llm(model_name:str, temperature=0.0):


    llm = ChatOpenAI(model=model_name, 
                     base_url=API_BASE_URL, 
                     api_key=LITELLM_MASTER_KEY, 
                     temperature=temperature, 
    )

    return llm


In [6]:

m="claude-3-haiku-20240307"
# m="llama3.2:3b-instruct-q8_0"
# m="ollama/deepseek-coder-v2:16b-lite-instruct-q8_0"
# m="ollama/llama3.2:3b-instruct-q8_0"

llm,_ = get_llm(m)
print(f"llm: {llm}")

llm: callbacks=<langchain_core.callbacks.manager.CallbackManager object at 0x71c9a40784c0> client=<openai.resources.chat.completions.Completions object at 0x71c9a407a7d0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x71c9a40b90c0> root_client=<openai.OpenAI object at 0x71c9b06e59f0> root_async_client=<openai.AsyncOpenAI object at 0x71c9a407baf0> model_name='claude-3-haiku-20240307' temperature=0.0 openai_api_key=SecretStr('**********') openai_api_base='http://192.168.30.100:8000/' openai_proxy=''


In [7]:
# Create a simple agent
agent = Agent(
    role='Greeter',
    goal='Greet the world with enthusiasm',
    backstory='You are an AI assistant designed to greet people warmly.',
    verbose=True,
    llm=llm,
    #llm=llm,
)


In [8]:

poet = Agent(
    role='poet',
    goal='Write a Haiku',
    backstory='You are an AI assistant designed to write haikus.',
    verbose=True,
    llm=llm,
)
# Create a simple task
task = Task(
    description='Create a warm greeting for the world',
    expected_output='Hello, world!',
    agent=agent
)

poem = Task(
    description="write a haiku",
    expected_output=" a haiku poem",
    agent=poet
)

In [9]:
# Create the crew
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    task_callback=task_callback_function,
    
)
# Run the crew
result = crew.kickoff()

print("Crew Result:")
print(result)

# Agent: Greeter
## Task: Create a warm greeting for the world


# Agent: Greeter
## Final Answer: 
Hello, world! It is with great enthusiasm and joy that I greet you today. I am overjoyed to connect with all of you, near and far, and to share in the wonder of this moment. Each and every one of you is a precious and valued member of our global community, and I am honored to welcome you with open arms. 

May this greeting fill your heart with warmth and your spirit with hope. Let us come together in a spirit of unity, compassion, and celebration of our shared humanity. I extend my heartfelt wishes for peace, prosperity, and happiness to all. Hello, world - I am thrilled to greet you!


Task_Callback function called
task callback: Hello, world! It is with great enthusiasm and joy that I greet you today. I am overjoyed to connect with all of you, near and far, and to share in the wonder of this moment. Each and every one of you is a precious and valued member of our global community, and 